- `match_id`: 比赛的唯一标识符。
- `player1` 和 `player2`: 参与比赛的两位选手的名字。
- `elapsed_time`: 从比赛开始到当前点的经过的时间。
- `set_no`: 当前的比赛局数。
- `game_no`: 当前的比赛场数。
- `point_no`: 当前的比赛分数。
- `p1_sets` 和 `p2_sets`: 选手1和选手2赢得的比赛局数。
- `p1_games` 和 `p2_games`: 选手1和选手2赢得的比赛场数。
- `p1_points` 和 `p2_points`: 选手1和选手2赢得的比赛分数。
- `p1_break_pt_missed` 和 `p2_break_pt_missed`: 选手1和选手2错过的破发点。
- `p1_distance_run` 和 `p2_distance_run`: 选手1和选手2在比赛中跑过的距离。
- `rally_count`: 一次交换球的次数。
- `speed_mph`: 发球的速度，单位是英里每小时。
- `serve_width`: 发球的宽度。
- `serve_depth`: 发球的深度。
- `return_depth`: 返回球的深度。
- `Net-cnt`: 上网的次数。
- `Net-sc-rate`: 上网成功的比率。


In [164]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
df=pd.read_csv('Wimbledon_featured_matches.csv',index_col=0)
p1 = 0
p2 = 0
gameno = 1
index = -1

tot_break_game = 0
short_break_game = 0
pre_break_game = []

tot_hold_game = 0
short_hold_game = 0
pre_hold_game = []

match_t = 1
set_no = 1

tot_game_cnt = 0

tot_net_cnt = 0
tot_net_won = 0
for t,row in df.iterrows():
    # ------------------初始化计数器-----------------------------------------
    if gameno != int(row['game_no']):   #   换了一个game
        p1 = 0
        p2 = 0
        gameno = int(row['game_no'])
        tot_game_unf_err = 0
    if row['set_no'] == 1 and set_no != 1:  #   换了一个match
        set_no = 1
        # match_t = t
        tot_break_game = 0
        short_break_game = 0
        pre_break_game = []

        tot_hold_game = 0
        short_hold_game = 0
        pre_hold_game = []

        tot_game_cnt = 0
        tot_net_cnt = 0
        tot_net_won = 0
        
    # ------------------预处理得分情况 P1S P2S 本场game的得分领先进度 SGA-----------------------------------------
    win_no = int(row['point_victor'])
    p1 = p1 + (win_no == 1)
    p2 = p2 + (win_no == 2)
    df.loc[t, 'P1S'] = int(p1)
    df.loc[t, 'P2S'] = int(p2)
    df.loc[t,'SGA']= p1-p2
    
    #----------计算奔跑回报率 = 奔跑距离*(是否得分 + log2(1 + 2*分数比))/(1 + 奔跑距离)----------
    run = row['p1_distance_run']
    score_rate = 0
    if p2 != 0:
        score_rate =  p1/p2
    run_rate = run * ((row['point_victor'] == 1) +  math.log2(1 + 2 * score_rate ))/ (1 + run)
    df.loc[t,'RE'] = run_rate 
    
    
    #----------计算非强迫性失误----------------------------------------------------------------------
    if row['p1_unf_err'] == 1:
        tot_game_unf_err -= 1
    df.loc[t,'Unf_err_cnt'] =  tot_game_unf_err

    

    # ------------------预处理短期破球数等 STB LTB STH LTH-----------------------------------------
    df.loc[t,'STB'] = short_break_game
    df.loc[t,'LTB'] = tot_break_game
    df.loc[t,'STH'] = short_hold_game
    df.loc[t,'LTH'] = tot_hold_game
    tot_game_cnt += 1
    set_no = row['set_no']

    if row['p1_break_pt_won'] == 1:#    p1破发
        tot_break_game += 1
        short_break_game += 1
        pre_break_game.append(tot_game_cnt)

    elif row['game_victor'] == 1 and row['server'] == 1:#     p1保发
        tot_hold_game += 1
        short_hold_game += 1
        pre_hold_game.append(tot_game_cnt)

    for index, value in enumerate(pre_break_game):
        if (tot_game_cnt - 5 == value and value > 0):
            pre_break_game[index] = -1
            short_break_game -= 1

    for index, value in enumerate(pre_hold_game):
        if (tot_game_cnt - 5 == value and value > 0):
            pre_hold_game[index] = -1
            short_hold_game -= 1

    df.loc[t,'STB'] = short_break_game
    df.loc[t,'LTB'] = tot_break_game
    df.loc[t,'STH'] = short_hold_game
    df.loc[t,'LTH'] = tot_hold_game

    #------------------上网次数 Net-cnt-----------------------------------------

    if row['p1_net_pt'] == 1:
        tot_net_cnt +=1
    if row['p1_net_pt_won'] == 1:
        tot_net_won +=1

    df.loc[t,'Net-cnt'] = tot_net_cnt
    if tot_net_cnt != 0:
        df.loc[t,'Net-sc-rate'] = tot_net_won / tot_net_cnt
    else:
        df.loc[t,'Net-sc-rate'] = 0
        
    #------------------是否是发球方 BTS-----------------------------------------
    # print(row['server'] == 1)
    df.loc[t,'BTS']=int(row['server'] == 1)
    
    #------------------# 发球深度 SD-----------------------------------------
    if row['serve_depth'] == 'NCTL':
        df.loc[t,'SD'] = 3.5
    elif row['serve_depth'] == 'CTL':
        df.loc[t,'SD'] = 2
    #------------------# 发球深度 SW----------------------------------------
    if row['serve_width'] == 'B'or row['serve_width'] == 'C':
        df.loc[t,'SW'] = 0
    elif row['serve_width'] == 'BC':
        df.loc[t,'SW'] = 7.8
    elif row['serve_width'] == 'BW':
        df.loc[t,'SW'] = 23.4
    elif row['serve_width'] == 'W':
        df.loc[t,'SW'] = 31.2

    
df.to_csv('new_data.csv')

In [165]:
df = pd.read_csv('new_data.csv')
print(df.head())

              match_id         player1        player2 elapsed_time  set_no  \
0  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:00       1   
1  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:38       1   
2  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:01       1   
3  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:31       1   
4  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:02:21       1   

   game_no  point_no  p1_sets  p2_sets  p1_games  ...  Unf_err_cnt  STB  LTB  \
0        1         1        0        0         0  ...         -1.0  0.0  5.0   
1        1         2        0        0         0  ...         -1.0  0.0  5.0   
2        1         3        0        0         0  ...         -1.0  0.0  5.0   
3        1         4        0        0         0  ...         -1.0  0.0  5.0   
4        1         5        0        0         0  ...         -1.0  0.0  5.0   

   STH   LTH  Net-cnt  Net-sc-rate  BTS   SD    SW

In [166]:
df=pd.read_csv('new_data.csv',index_col=0)
count = 0
for t,row in df.iterrows():
    # 发球技术
    x1 = row['speed_mph'] # 发球速度
    x2 = row['SD'] # 发球深度
    x3 = row['SW'] # 发球宽度
    

    # 比赛表现
    x4 = row['STB'] # 5局短期破发数
    x5 = row['LTB'] # 长期破发数
    x6 = row['STH'] # 5局短期保发数
    x7 = row['LTH'] # 长期保发数

    # 个人技术表现
    x8 = row['Net-cnt'] # 上网次数
    x9 = row['Net-sc-rate'] # 上网得分率
    x10 = row['RE'] # 奔跑回报率
    x11 = row['Unf_err_cnt']   # 本场game是否出现非强迫失误

    # 比赛局势
    x12 = row['BTS'] == '1'  # 是否是发球方
    x13 = row['p1_games'] # 当前set中赢得的game数量
    x14 = row['SGA'] # 本场game的得分领先进度
    # 打印当前行和一些关键的变量
    print(f"Row {t}: x1={x1}, x2={x2}, x3={x3}, x4={x4}, x5={x5}, x6={x6}, x7={x7}, x8={x8}, x9={x9}, x10={x10}, x11={x11}, x12={x12}, x13={x13}, x14={x14}")

    # 更新计数器
    count += 1
    if count >= 5:  # 只打印前5个循环
        break

    
    
    

Row 2023-wimbledon-1301: x1=95.0, x2=3.5, x3=23.4, x4=0.0, x5=5.0, x6=1.0, x7=20.0, x8=25.0, x9=0.76, x10=2.46192630124206, x11=-1.0, x12=False, x13=0, x14=2.0
Row 2023-wimbledon-1301: x1=118.0, x2=3.5, x3=23.4, x4=0.0, x5=5.0, x6=1.0, x7=20.0, x8=25.0, x9=0.76, x10=2.46192630124206, x11=-1.0, x12=False, x13=0, x14=2.0
Row 2023-wimbledon-1301: x1=120.0, x2=3.5, x3=23.4, x4=0.0, x5=5.0, x6=1.0, x7=20.0, x8=25.0, x9=0.76, x10=2.46192630124206, x11=-1.0, x12=False, x13=0, x14=2.0
Row 2023-wimbledon-1301: x1=130.0, x2=3.5, x3=23.4, x4=0.0, x5=5.0, x6=1.0, x7=20.0, x8=25.0, x9=0.76, x10=2.46192630124206, x11=-1.0, x12=False, x13=0, x14=2.0
Row 2023-wimbledon-1301: x1=112.0, x2=3.5, x3=23.4, x4=0.0, x5=5.0, x6=1.0, x7=20.0, x8=25.0, x9=0.76, x10=2.46192630124206, x11=-1.0, x12=False, x13=0, x14=2.0


In [167]:
from sklearn.preprocessing import MinMaxScaler
# ------------------------------------------数据归一化---------------------------------------

df=pd.read_csv('new_data.csv',index_col=0)
# 选择归一化的列
columns_to_normalize = ['speed_mph', 'SD', 'SW', 'STB', 'LTB', 'STH', 'LTH', 'Net-cnt', 'Net-sc-rate', 'RE', 'Unf_err_cnt', 'BTS', 'p1_games', 'SGA']

# 创建归一化器
scaler = MinMaxScaler()

# 对选定的列进行归一化
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# 保存特定的列到新的 CSV 文件
df[columns_to_normalize].to_csv('normalization_data.csv', index=False)

In [171]:
from sklearn.preprocessing import MinMaxScaler
# ------------------------------------------数据归一化---------------------------------------

df=pd.read_csv('check.csv',index_col=0)
# 选择归一化的列
columns_to_normalize = ['speed_mph', 'SD', 'SW', 'STB', 'LTB', 'STH', 'LTH', 'Net-cnt', 'Net-sc-rate', 'RE', 'Unf_err_cnt', 'BTS', 'p1_games', 'SGA']

# 创建归一化器
scaler = MinMaxScaler()

# 对选定的列进行归一化
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# 保存特定的列到新的 CSV 文件
df[columns_to_normalize].to_csv('normalization_check_data.csv', index=False)

In [168]:
import pandas as pd
import numpy as np

df=pd.read_csv('normalization_data.csv')
print("发球技术")
# 选择用于熵权法的列
columns_to_use = ['speed_mph', 'SD', 'SW']
data = df[columns_to_use]

# 计算每个指标的比重
proportions = data / data.sum()

# 计算熵值
proportions = proportions + 1e-10
entropy = -np.sum(proportions * np.log(proportions), axis=0) / np.log(len(data))

# 计算信息熵冗余度
redundancy = 1 - entropy

# 计算权重
weights = redundancy / redundancy.sum()

# 显示权重
print(weights)
# ---------------------------------------------------------------------------------------------------
print("比赛表现")
# 选择用于熵权法的列
columns_to_use = ['STB', 'LTB', 'STH', 'LTH']
data = df[columns_to_use]

# 计算每个指标的比重
proportions = data / data.sum()

# 计算熵值
proportions = proportions + 1e-10
entropy = -np.sum(proportions * np.log(proportions), axis=0) / np.log(len(data))

# 计算信息熵冗余度
redundancy = 1 - entropy

# 计算权重
weights = redundancy / redundancy.sum()

# 显示权重
print(weights)
# ---------------------------------------------------------------------------------------------------
print("个人技术表现")
# 选择用于熵权法的列
columns_to_use = [ 'Net-cnt', 'Net-sc-rate', 'RE', 'Unf_err_cnt']
data = df[columns_to_use]

# 计算每个指标的比重
proportions = data / data.sum()

# 计算熵值
proportions = proportions + 1e-10
entropy = -np.sum(proportions * np.log(proportions), axis=0) / np.log(len(data))

# 计算信息熵冗余度
redundancy = 1 - entropy

# 计算权重
weights = redundancy / redundancy.sum()

# 显示权重
print(weights)
# ---------------------------------------------------------------------------------------------------
print("比赛局势")
# 选择用于熵权法的列
columns_to_use = ['BTS', 'p1_games', 'SGA']
data = df[columns_to_use]

# 计算每个指标的比重
proportions = data / data.sum()

# 计算熵值
proportions = proportions + 1e-10
entropy = -np.sum(proportions * np.log(proportions), axis=0) / np.log(len(data))

# 计算信息熵冗余度
redundancy = 1 - entropy

# 计算权重
weights = redundancy / redundancy.sum()

# 显示权重
print(weights)

发球技术
speed_mph    0.163049
SD           0.406377
SW           0.430574
dtype: float64
比赛表现
STB    0.679110
LTB    0.103072
STH    0.180335
LTH    0.037482
dtype: float64
个人技术表现
Net-cnt        0.413801
Net-sc-rate    0.128662
RE             0.335820
Unf_err_cnt    0.121717
dtype: float64
比赛局势
BTS         0.502218
p1_games    0.323121
SGA         0.174661
dtype: float64


|              | 发球技术 | 比赛表现 | 个人技术表现 | 比赛局势 |
|--------------|----------|----------|--------------|----------|
| 发球技术      | 1        | 2        | 3            | 4        |
| 比赛表现      | 1/2      | 1        | 2            | 3        |
| 个人技术表现  | 1/3      | 1/2      | 1            | 2        |
| 比赛局势      | 1/4      | 1/3      | 1/2          | 1        |


In [169]:
from scipy.linalg import eig
from tabulate import tabulate

# 判断矩阵
judgment_matrix = np.array([
    [1, 2, 3, 4],
    [1/2, 1, 2, 3],
    [1/3, 1/2, 1, 2],
    [1/4, 1/3, 1/2, 1]
])

# 计算特征值和特征向量
eigenvalues, eigenvectors = eig(judgment_matrix)

# 提取最大特征值对应的特征向量
max_eigenvalue = max(eigenvalues.real)
max_eigenvector = eigenvectors[:, eigenvalues.argmax()]

# 归一化特征向量以得到权重
weights = max_eigenvector / np.sum(max_eigenvector)

# 计算CI（一致性指标）
n = judgment_matrix.shape[0]  # 判断矩阵的阶数
CI = (max_eigenvalue - n) / (n - 1)

# RI值（对于不同阶数的矩阵，RI值是预先确定的）
RI_dict = {1: 0, 2: 0, 3: 0.58, 4: 0.9, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}
RI = RI_dict[n]

# 计算CR（一致性比率）
CR = CI / RI

# 定义类别
categories = ['发球技术', '比赛表现', '个人技术表现', '比赛局势']

# 创建一个 DataFrame 来存储结果，并将 'Category' 列设置为索引
result = pd.DataFrame({
    'Weights': weights.real,
    'CI': [CI]*len(categories),
    'CR': [CR]*len(categories)
}, index=categories)

# 将数值转换为百分数，并保留六位小数
result = result.applymap(lambda x: '{:.6%}'.format(x))

# 输出结果
print(tabulate(result, headers='keys', tablefmt='psql', showindex=True))

+--------------+------------+-----------+-----------+
|              | Weights    | CI        | CR        |
|--------------+------------+-----------+-----------|
| 发球技术     | 46.729598% | 1.032783% | 1.147537% |
| 比赛表现     | 27.718059% | 1.032783% | 1.147537% |
| 个人技术表现 | 16.008848% | 1.032783% | 1.147537% |
| 比赛局势     | 9.543495%  | 1.032783% | 1.147537% |
+--------------+------------+-----------+-----------+
